In [7]:
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
import pandas as pd

In [11]:
import pandas as pd
df = pd.read_csv("D:/datasets/Sentiment_dataset.csv", encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])
texts = df['text'].astype(str).tolist()[:10000]
labels = df['target'].apply(lambda x: 0 if x == 0 else 1).tolist()[:10000]

In [15]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)

In [17]:
dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), labels)).shuffle(1000).batch(8)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

In [25]:
from transformers import AdamWeightDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = AdamWeightDecay(learning_rate=5e-5)
loss_fn = SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
model.fit(train_dataset, epochs=2)

Epoch 1/2
85/85 [==============================] - 56s 453ms/step - loss: 2.5523e-05 - accuracy: 1.0000
Epoch 2/2
85/85 [==============================] - 39s 464ms/step - loss: 1.8469e-06 - accuracy: 1.0000


In [26]:
loss, accuracy = model.evaluate(test_dataset)

22/22 [==============================] - 6s 74ms/step - loss: 7.1526e-07 - accuracy: 1.0000


In [29]:
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
for batch in test_dataset.take(1):
    preds = model.predict(batch[0]).logits
    pred_labels = tf.argmax(preds, axis=1).numpy()
    for i in range(5):
        print(f"Text: {tokenizer.decode(batch[0]['input_ids'][i], skip_special_tokens=True)}")
        print(f"Predicted label: {pred_labels[i]}, True label: {batch[1][i].numpy()}")

Test Loss: 7.152556236178498e-07, Test Accuracy: 1.0
1/1 [==============================] - 4s 4s/step
Text: nan
Predicted label: 1, True label: 1
Text: nan
Predicted label: 1, True label: 1
Text: nan
Predicted label: 1, True label: 1
Text: nan
Predicted label: 1, True label: 1
Text: nan
Predicted label: 1, True label: 1
